
# カタカナから漢字・アルファベットを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformerで，スクラッチで学習する。

入力の文字（カタカナ）は一文字ずつに分割，また，出力の文字（漢字・アルファベットなど）も一文字ずつに分割して，学習する。

データセットは，漢字からカタカナを推測するモデルのデータを，漢字とカナを逆にした者を作成して利用する。

## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，

In [1]:
# スクリプトで使う環境変数をセット
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        ver=l.rstrip()
        break
os.environ["VER"]=ver
os.environ['DATA']='../dict/data/data.'+os.environ["VER"]
os.environ['DIR']='dataset/dataset.'+os.environ["VER"]
os.environ['LOG']='logs/logs_r.'+os.environ["VER"]
os.environ['MODEL']='model/model_r.'+os.environ["VER"]


In [2]:
!echo $VER

1.7o


In [ ]:
# 訓練実行
!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 64 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --source_lang kana \
  --target_lang kanji \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


[00:00:09] Pre-processing sequences       ██████████████████ 0        /        0[00:00:00] Pre-processing sequences       ██████████████████ 0        /        0/home/analysis01/src/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
num_epochs:100
/home/analysis01/src/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/functional.py:6041: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch: 1, Train loss: 3.988, Val loss: 2.830, Epoch time = 7938.711s
Epoch: 2, Train loss: 2.692, Val loss: 1.898, Epoch time = 7915.042s
Epoch: 3, Train loss: 1.865, Val loss: 1.200, Epoch time = 7901.388s
Epoch: 4, Train loss: 1.395, Val loss: 1.006, Epoch time = 7902.069s
Epoch: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/logs_r.$VER

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file    
jsonl形式の検証用データファイル

- model_file    
transformer_model.pyで作成したモデルファイル。

- outfile    
検証結果を格納するファイル

- device    
cpu限定

- nbest    
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width    
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len    
出力するカタカナ姓名の最大長さ

- search    
greedy,beam,bothから選択    
  - greedy    
貪欲法による探索    
  - beam    
ビームサーチによる探索    
  - both    
貪欲砲，ビームサーチ両方の探索    

In [2]:
# 検証用データを実行
!python ./generate.py \
    --test_file $DIR/test.jsonl \
    --model_file $MODEL/checkpoint_best.pt \
    --outfile $DIR/generate_r.txt \
    --source_lang kana \
    --target_lang kanji \
    --device cpu \
    --nbest 5 \
    --beam_width 5 \
    --max_len 100 \
    --search beam


/home/analysis01/src/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
# 検証結果を評価
!python evaluate.py --infile $DIR/generate_r.txt


dataset/dataset.1.7o/generate_r.txt,acc=0.6927381865776052


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。


In [4]:
# カナから漢字
!python conv_scripted.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_r_$VER.pt \
    --encoder=$MODEL/encoder_r_$VER.pt \
    --decoder=$MODEL/decoder_r_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_r_$VER.pt \
    --generator=$MODEL/generator_r_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_r_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_r_$VER.pt \
    --vocab_src=$MODEL/vocab_src_r_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_r_$VER.txt \
    --params=$MODEL/params_r_$VER.json \
    --device=cpu

ver=1.7o
/home/analysis01/src/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/analysis01/src/kanjikana-model/venv/lib/python3.12/site-packages/torch/jit/_recursive.py:313: UserWarning: 'norm' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(


In [5]:
!date

2025年 10月  8日 水曜日 16:55:29 JST
